In [1]:
!pip install hyperopt

In [13]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

import os
import datetime

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_road_signs'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_road_signs


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')
signs = pd.read_csv('data/signnames.csv')

In [5]:
x_train, y_train, x_test, y_test = train['features'], train['labels'], test['features'], test['labels']

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((34799, 32, 32, 3), (34799,), (4410, 32, 32, 3), (4410,))

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = x_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, x_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  log_dir = os.path.join('logs', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir, histogram_freq=1)

  model.fit(
      x_train, 
      y_train,
      batch_size = params_fit.get('batch_size', 128),
      epochs= params_fit.get('epochs', 5),
      verbose = params_fit.get('verbose', 1),
      validation_data = params_fit.get('validation_data', (x_train, y_train)),
      callbacks = [tensorboard_callback]
      )
  
  return model

def predict(model_trained, x_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis =1)

  y_pred_prob = model_trained.predict(x_test)
  y_pred = np.argmax(y_pred_prob, axis = 1)
  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_model_v5(input_shape, num_classes):
  return Sequential([
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape= input_shape ),
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
                    MaxPool2D(),
                    Dropout(0.3),

                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(0.3),      

                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(0.3),                

                    Flatten(),

                    Dense(1024, activation='relu'),
                    Dropout(0.3),

                    Dense(1024, activation='relu'),
                    Dropout(0.3),

                    Dense(num_classes, activation='softmax')

                    ])

In [10]:
def train_and_predict(model):
  model_trained = train_model(model, x_train, y_train)
  return predict(model_trained, x_test, y_test)

train_and_predict(get_cnn_model_v5(input_shape, num_classes))

Epoch 1/5
272/272 [==============================] - 5s 19ms/step - loss: 2.5760 - accuracy: 0.3012 - val_loss: 1.0317 - val_accuracy: 0.6946
Epoch 2/5
272/272 [==============================] - 5s 17ms/step - loss: 0.9636 - accuracy: 0.6935 - val_loss: 0.3002 - val_accuracy: 0.9014
Epoch 3/5
272/272 [==============================] - 5s 17ms/step - loss: 0.4705 - accuracy: 0.8514 - val_loss: 0.1303 - val_accuracy: 0.9607
Epoch 4/5
272/272 [==============================] - 5s 17ms/step - loss: 0.3187 - accuracy: 0.9003 - val_loss: 0.0646 - val_accuracy: 0.9797
Epoch 5/5
272/272 [==============================] - 5s 18ms/step - loss: 0.2401 - accuracy: 0.9271 - val_loss: 0.0661 - val_accuracy: 0.9805


0.9448979591836735

In [20]:
def get_model(params):
  return Sequential([
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape= input_shape ),
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_one']),

                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_two']),      

                    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
                    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_three']),                

                    Flatten(),

                    Dense(1024, activation='relu'),
                    Dropout(params['dropout_dense_block_one']),

                    Dense(1024, activation='relu'),
                    Dropout(params['dropout_dense_block_two']),

                    Dense(num_classes, activation='softmax')

                    ])

ERROR! Session/line number was not unique in database. History logging moved to new session 59


In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  log_dir = os.path.join('logs', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir, histogram_freq=1)

  model.fit(
      x_train, 
      y_train,
      batch_size = int(params.get('batch_size', 128)),
      epochs= 5,
      verbose = 0
      )
  
  score = model.evaluate(x_test, y_test, verbose =0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))
  
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [30]:
space = {
    'batch_size' : hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3,0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3,0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3,0.5),    
    
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3,0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3,0.7)

}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    25,
    Trials()
)

{'batch_size': 160.0, 'dropout_cnn_block_one': 0.4092578061434435, 'dropout_cnn_block_three': 0.471545826236927, 'dropout_cnn_block_two': 0.36653884963619193, 'dropout_dense_block_one': 0.3232028556260617, 'dropout_dense_block_two': 0.31364313922019244}
accuracy=0.9299319982528687
{'batch_size': 200.0, 'dropout_cnn_block_one': 0.32563850096308805, 'dropout_cnn_block_three': 0.3226934304209245, 'dropout_cnn_block_two': 0.4441498963396938, 'dropout_dense_block_one': 0.41506932756046555, 'dropout_dense_block_two': 0.6559464825213805}
accuracy=0.9607709646224976
{'batch_size': 100.0, 'dropout_cnn_block_one': 0.48254097442606386, 'dropout_cnn_block_three': 0.37290540771835506, 'dropout_cnn_block_two': 0.39815313421153664, 'dropout_dense_block_one': 0.6678885600731449, 'dropout_dense_block_two': 0.5778436298497875}
accuracy=0.9181405901908875
{'batch_size': 130.0, 'dropout_cnn_block_one': 0.4491255867554854, 'dropout_cnn_block_three': 0.3363874024607424, 'dropout_cnn_block_two': 0.3863109563